In [1]:
i0=1
i1=1
i2=1
i3=1

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
from tqdm import tqdm
import sys
sys.path.append('/home/icb/manuel.gander/Atl/notebooks/')
import utils
utils = importlib.reload(utils)
import scipy
import time

import warnings
from scipy.stats import ConstantInputWarning
from sklearn.exceptions import ConvergenceWarning

# Ignore ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
seeds = [0,1,2,3]
frs = [0.25, 0.5, 0.75, 1.0]
kn=10**9
reps=100

In [3]:
alpha = alphas[i0]
l1_ratio = l1_ratios[i1]
seed = seeds[i2]
fr = frs[i3]

dataset = 'RNA'
source = 'GDSC2'

In [4]:
D_prot, features, celllines = utils.load_dataset(dataset)
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']==source].copy()
dfv = dfv[np.isfinite(dfv['AUC'])].copy()

100%|███████████████████████████████████████████████████████████████████████████████| 1960464/1960464 [00:01<00:00, 1325800.54it/s]


In [5]:
vc = dfv['PubChem_CID'].value_counts()
drugs = vc[vc>150].index 
dfv = dfv[dfv['PubChem_CID'].isin(drugs)].copy()
ns=[]
ds=[]
for d,df in list(dfv[['PubChem_CID', 'Cello', 'sensitivity']].groupby('PubChem_CID')):
    ns.append((df['sensitivity']=='yes').sum())
    ds.append(d)
dfd = pd.DataFrame({'PubChem_CID':ds, 'n_sensitive':ns})
drugs = list(dfd[dfd['n_sensitive']>25]['PubChem_CID'])


ccls = sorted(set(dfv['Cello']))
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))
dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()

628


In [ ]:
frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps)
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['source']=source
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['seed']=seed
Df['fr']=fr
Df['kn']=kn

Df.to_pickle(f'/home/icb/manuel.gander/Atl/data/dones_sub_new/{alpha}_{l1_ratio}_{seed}_{fr}.pkl')

In [ ]:
frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    res['td']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps, source0='all')
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                t0=time.time()
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                td = time.time()-t0
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))
                res['td'].append(td)

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['source']=source
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['kn'] = kn
Df['n_pca']=n_components

In [ ]:
D_prot, features, celllines = utils.load_dataset(dataset)
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']==source].copy()

ccls = sorted(set(dfv['Cello']))
print(len(ccls))
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))

dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()
drugs = sorted(set(dfv['PubChem_CID']))


frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps)
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['source']=source
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['seed']=seed
Df['fr']=fr
Df['kn']=kn

Df.to_pickle(f'/home/icb/manuel.gander/Atl/data/dones_sub_new/{alpha}_{l1_ratio}_{seed}_{fr}.pkl')



In [3]:
seeds = [0,1,2,3]
frs = [0.25,0.5,0.75,1]

In [4]:
seed = seeds[i2]
fr = frs[i3]

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import importlib
from tqdm import tqdm
utils = importlib.reload(utils)
import scipy

import warnings
from scipy.stats import ConstantInputWarning
from sklearn.exceptions import ConvergenceWarning

# Ignore ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

D_prot, features, celllines = utils.load_dataset('RNA')
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']=='GDSC2'].copy()

ccls = sorted(set(dfv['Cello']))
print(len(ccls))
seed = 0
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))

dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()

vc = dfv['PubChem_CID'].value_counts()
drugs = vc[vc>150].index 
dfv = dfv[dfv['PubChem_CID'].isin(drugs)].copy()

ns=[]
ds=[]
for d,df in list(dfv[['PubChem_CID', 'Cello', 'sensitivity']].groupby('PubChem_CID')):
    ns.append((df['sensitivity']=='yes').sum())
    ds.append(d)
dfd = pd.DataFrame({'PubChem_CID':ds, 'n_sensitive':ns})
drugs = list(dfd[dfd['n_sensitive']>25]['PubChem_CID'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2233583/2233583 [00:01<00:00, 1571169.26it/s]


628


In [ ]:
# Only keep drugs with more than 500 cell lines

In [21]:
alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
kn=200
reps=100

In [22]:
alpha = alphas[i2]
l1_ratio = l1_ratios[i3]

In [ ]:
0.25

In [23]:
frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps, source0='all')
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio

 29%|███████████████████████████████████████▌                                                                                                  | 68/237 [06:37<16:28,  5.85s/it]

KeyboardInterrupt



In [1]:
i0=0
i1=0
i2=0
i3=0


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import importlib
from tqdm import tqdm
utils = importlib.reload(utils)
import scipy


import warnings
from scipy.stats import ConstantInputWarning
from sklearn.exceptions import ConvergenceWarning

# Ignore ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

datasets=['kinase_scores', 'RNA', 'atl_full_and_kinases', 'identity']

sources=['GDSC2', 'GDSC1', 'CTD2', 'DTP', 'CTPR', 'MR_NCI60']


alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
seeds = [0,1,2,3]
frs = [0.25,0.5,0.75,1]

alpha = alphas[i0]
l1_ratio = l1_ratios[i1] 
seed = seeds[i2]
fr = frs[i3]

kn=200
reps=100



D_prot, features, celllines = utils.load_dataset('RNA')
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']=='GDSC2'].copy()

ccls = sorted(set(dfv['Cello']))
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))

dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()

vc = dfv['PubChem_CID'].value_counts()
drugs = vc[vc>150].index 
dfv = dfv[dfv['PubChem_CID'].isin(drugs)].copy()

ns=[]
ds=[]
for d,df in list(dfv[['PubChem_CID', 'Cello', 'sensitivity']].groupby('PubChem_CID')):
    ns.append((df['sensitivity']=='yes').sum())
    ds.append(d)
dfd = pd.DataFrame({'PubChem_CID':ds, 'n_sensitive':ns})
drugs = list(dfd[dfd['n_sensitive']>25]['PubChem_CID'])



frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps, source0='all')
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['Source']='GDSC2'
Df['data']='RNA'
Df['fraction']=fr
Df['seed']=seed

Df.to_pickle(f'/home/icb/manuel.gander/Atl/data/dones_sub/{fr}_{alpha}_{l1_ratio}_{seed}.pkl')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2233583/2233583 [00:01<00:00, 1482561.85it/s]
0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [15]:
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']=='GDSC2'].copy()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2233583/2233583 [00:01<00:00, 1507703.64it/s]


In [16]:
vc = dfv['PubChem_CID'].value_counts()
drugs = vc[vc>150].index 
dfv = dfv[dfv['PubChem_CID'].isin(drugs)].copy()

ns=[]
ds=[]
for d,df in list(dfv[['PubChem_CID', 'Cello', 'sensitivity']].groupby('PubChem_CID')):
    ns.append((df['sensitivity']=='yes').sum())
    ds.append(d)
dfd = pd.DataFrame({'PubChem_CID':ds, 'n_sensitive':ns})
drugs = list(dfd[dfd['n_sensitive']>25]['PubChem_CID'])

ccls = sorted(set(dfv['Cello']))
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))
dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()

In [18]:
ccls = sorted(set(dfv['Cello']))
np.random.seed(seed)
ccls_sel = np.random.choice(ccls, int(len(ccls)*fr))
dfv = dfv[dfv['Cello'].isin(ccls_sel)].copy()

In [19]:
dfv

,PubChem_CID,Cello,Source,AUC,ic50,min_dose,max_dose,Dr_repr_name,Cello_i,sensitivity
46063,10127622,8305C,GDSC2,0.745760,2.554436,-32,24,selumetinib,8305C_686,not
46072,1018,8305C,GDSC2,1.016705,4.001542,-19,10,picolinic-acid,8305C_686,not
46076,10184653,8305C,GDSC2,0.732429,0.777700,-27,18,afatinib,8305C_686,not
46100,10384072,8305C,GDSC2,0.914678,0.837892,-21,10,Wee1 Inhibitor,8305C_686,not
46102,10385095,8305C,GDSC2,0.763979,0.573540,-30,15,Telomerase Inhibitor IX,8305C_686,not
...,...,...,...,...,...,...,...,...,...,...
2210991,THR-101,WM793,GDSC2,1.051011,1.034110,-19,10,THR-101,WM793_782,not
2210992,THR-102,WM793,GDSC2,0.944337,3.994989,-19,10,THR-102,WM793_782,not
2210993,THR-103,WM793,GDSC2,0.946457,1.024747,-19,10,THR-103,WM793_782,not
2211003,VTP-A,WM793,GDSC2,1.017697,0.077528,-29,0,VTP-A,WM793_782,not
